# Hep Recommender
> A recommender system for scientific articles in the field of High Energy Physics.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/hep_recommender.png

# Introduction

In this note I want to discuss [hep-recommender](https://dashboard.heroku.com/apps/hep-recommender), a recommender system for scientific papers in the field of High Energy Physics (HEP) aimed to help researchers in the exploration of relevant literature.  This is a personal project I have been working recently in collaboration with [José Eliel Camargo Molina](https://github.com/JoseEliel).    A [submission](https://devpost.com/software/hep-recommender) of this project to the [Facebook Artificial Intelligence Hackathon (2020)](https://fbai1.devpost.com) won the third place.


There are different approaches to determine the degree of similarity between articles in order to identify related work. Some works have used text-mining and natural language processing methods. Another popular approach is based on citation analysis, where the similarity between two articles is estimated based on bibliographic information [1,2,4,5]. Finally, other works have used a combination of the previous approaches, see for instance [3].    

In this project, we develop a recommender system based on citation analysis. More specifically, we will develop a recommender system that elaborates on the ideas of co-citation analysis [1,4] and co-citation proximity analysis (CPA) [2]. Co-citation analysis is based on the premise that articles which are frequently cited together (by the same papers) should be related to each other. CPA extends this idea by incorporating the notion that the closer the citations are to each other within the article text, the more likely it is that they are related. While these methods are relatively simple they provide a high quality of related article recommendations.   Our recommender system relies on a distributed representation of articles obtained by training a Skip-Gram model on reference lists.  This model also captures the notion that articles cited close to each other on the text are similar. 

In this note I want to go to some detail about the model powering our recommender system, how it compares to other approaches, and how we made it available via a web application.  

# Related work

I would like to start discussing some of the standard methods in the literature to quantify the similarity between two articles.  I will define the following concepts: 

* bibliometric coupling

* co-citation count
 
* co-citation proximity analysis (CPA)

Bibliometric couplign reflects the idea that two papers sharing a large portion of their references should be similar [5].  Co-citation count assigns a high degree of similarity to articles which are frequenly cited together (regardless of where the citations occcured within the text) [1].   CPA extends co-citation count by assigning more weigth to cases where the articles are cited close to each other within the text [2].  In general CPA is expected to give better results than co-citation count, though it requires some work in order to parse the references of each article keeping information about the location of the references within the text.
 
Lets formalize these concepts.  We consider a set of papers $\{  \omega_1, \omega_2, \ldots \omega_n  \}$. We define the quantities $c_{i,j}$, where $c_{i,j}=1$ if $\omega_i$ is cited by $\omega_j$ and $c_{i,j}=0$ otherwise.  Then we are ready to introduce our three measures of similarity:


* bibliographic coupling is defined as $\sum_{k=1}^{n}   c_{k,i}   c_{k,j}$



* co-citation count is defined as $\sum_{k=1}^{n}   c_{i,k}   c_{j,k}$



* CPA can be used to define a measure $\sum_{k=1}^{n} \eta_{i,j}^{k}  c_{i,k}   c_{j,k}$, where $n_{ij}^{k}$ are coefficients that penalize cases where the citations occur far from each other within the text.    

In the original CPA article [2], $\eta$ was fixed to $1$ if the citations occur in the same sentence, $1/2$ if they only occur in the same paragraph, $1/4$ if they only occur in the same chapter, and $1/8$ if they only appear on the same article. Parametric representations of $\eta$ have also been proposed [7]. 

# Distributed representation on co-citations


We propose a model that relies on the same idea behind CPA, namely, that the proximity of references within an article provide valuable information regarding the similarity of two articles.    However, we wanted to simplify the process of extracting the necessary data as much as possible.  


We assume that it is possible to extract the list of references of each article in the order of appearance within the text. For instance the following article  


> Higgs boson pair production in gluon fusion is the most promising process to find out whether the Higgs boson self-coupling is Standard-Model-like. Early studies of Higgs boson pair production within an EFT framework can be found in Refs. [1–3]. Many phenomenological investigations about the potential of this process to reveal New Physics have been performed since, see e.g. Refs. [4–8]...

would give rise to a reference list [1,2,3,4,5,6,7,8,...].   In this way, articles which are frequently close to each other on these reference lists should be very similar, as the community is citing them close to each other within the text of the articles.   

We will use an approach that has proven to be very fruitful in Natural Language Processing (NLP).  We will take the lists of references and train a Skip-Gram model, such that articles which tend to be cited close to each other will have similar embeddings.    


Our dataset consists of ordered lists of references.  Suppose we have a total of $n$ unique articles in our dataset and for each of these articles we have its reference list.  The Skip-gram model maximizes the following objective function:

\begin{align}
\frac{1}{n} \sum_{q=1}^{n}    \left[  \frac{1}{n_q}   \sum_{i=1}^{n_q}  \sum_{ -c \leq j  \leq c ,\, j\neq 0  } \log p(w_{i+j}^{q}| w_i^{q}) \right]
\end{align} 

Here $\omega_{i}^{q}$ represents the embedding of the $i$-th article appearing on the reference list of article $q$; $n_q$ represents the size of the reference list for article $q$, and $c$ represents the context-window size of the Skip-Gram model.  The probability is modelled using  a softmax function
 
\begin{align}
p(    \omega_{i}  |    \omega_j  )   = \frac{   \exp(   \omega_{i} \cdot \omega_{j}  )}{     \sum_{l=1}^{n}   \exp(   \omega_{l}  \cdot \omega_j   )   }
\end{align}

Just like in the Skip-Gram model used in NLP, in order to make the model training more efficient we need to implement negative sampling or hierarchical softmax.      A good implementation of the Skip-Gram model is provided by the [Gensim](https://radimrehurek.com/gensim/auto_examples/index.html) library for instance.

In order to explore the structure of the trained embeddings, I took a sample of articles and made a t-SNE visualization of their embeddings in two dimensions:

<img src="../images/tsne.png" width="400">

For this, I picked a set of articles that were published on the [arXiv](https://arxiv.org).  Each arXiv category was displayed with a different color.  On this plot, some of the arXiv categories that appear are: astro-ph (red), hep-ex (blue), hep-th (black) and hep-ph (green), among others.  As expected, the embeddings form cluster around their categories.

# Data

Regarding open acces digital libraries, the research community in the field of High Energy Physics mainly uses the [INSPIRE-HEP](https://inspirehep.net) and the [arXiv](https://arxiv.org). INSPIRE-HEP provides an API from which data for articles in the field of HEP can be retrieved, we use this API to extract the data we need.  Below I show some exploratory example.  Lets first define some classes that talk to the API and handle the response.

In [1]:
import requests
from typing import List, Dict


class InspireAPI:
    """
    Simple wrapper class around the INSPIRE API

    https://inspirehep.net

    methods:
       literature: gives access to the literature endpoint
    """

    LITERATURE = "https://inspirehep.net/api/literature/"

    def __init__(self):
        pass

    def literature(self, record_id: str):
        """
        Returns api response for a given record_id
        """
        url = self.LITERATURE + record_id
        return LiteratureRecord(requests.get(url).json())


class LiteratureRecord:
    """
    Datamodel class for handling literature record data,
    implementing basic methods to access the properties
    """

    def __init__(self, data: Dict):
        self.data = data
        
    @property 
    def record_id(self) -> str:
        """
        Returns the INSPIRE id of the article
        """
        return self.data['id']
        
    @property
    def metadata(self) -> Dict:
        """
        Returns article metadata
        """
        return self.data['metadata']

    @property
    def references(self) -> List[str]:
        """
        Returns reference list of the article as a List of INSPIRE article ids
        """
        if self.metadata.get("references"):
            return [
                element["record"]["$ref"].split("/")[-1]
                for element in self.metadata["references"]
                if element.get("record")
            ]    

We can now look at one example. All the bibliographic data is contained on the *metadata* property of our LiteratureRecord class.  I implemented a *references* property in order to retrieve the list of references (only the article identifiers).   Lets extract data for the article with the identifier ['11883'](https://labs.inspirehep.net/literature/11883),

In [2]:
inspireapi = InspireAPI()

record = inspireapi.literature('11883')

record.references

['40440',
 '12289',
 '12290',
 '14006',
 '12291',
 '12288',
 '9159',
 '43800',
 '43801']

In this way we are able to extract the lists of references we need for our model.  Note that the INSPIRE-API returns references in the order of appearance on the reference section of the article, and by tradition, the HEP community orders references by the order of appearance on the text. 

# Recommendations

We can use the Skip-Gram model embeddings and some metric on the vector space to provide recommendations for articles with at least some citations.  Very recent articles or unpopular articles with no citations will not have embeddings and we must find other method to provide recommendations for these articles.   For this reason, we consider two scenarios when we want to provide recommendations for a given article: 

*i)* The article has embeddings produced by our Skip-Gram model.  


*ii)* The article has no embeddings from the Skip-Gram model. 


In case *i)* we retrieve the top similar articles using cosine similarity as the metric.  Cosine similarity is defined as $\cos \theta_{12} = \hat \omega_{1} \cdot \hat \omega_{2}$, where $\hat \omega = \omega/|\omega|$ is a unit norm vector.   In this case we retrieve as recommendations the articles whose embedding have a smaller angle difference with the original article.  


In case *ii)*, we dont have an embedding but we can build one out of the references of the article.  We retrieve the references of the article by calling the INSPIRE-HEP api and then take the average vector $\omega_{\rm{avg}}^q =  1/n_q \sum_{i=1}^{n_q} \omega_i^{q}$ of the references. We can then proceed to retrieve the top similar articles using cosine similarity as before.    

These two simple methods are able to provide recommendations for most articles.   

# Web application

The recommender system has been made available at [hep-recommender](https://dashboard.heroku.com/apps/hep-recommender).   It is a Flask web application
currently hosted on Heroku with a gunicorn server. Storage of the model artifacts is done in AWS S3.

# Discussion

To see one example of the application, we can look the recommendations of similar articles for:

["Broken Symmetries and the Masses of Gauge Bosons"](https://labs.inspirehep.net/literature/11883) by Higgs, Peter W. (1964).


This article is one of the works behind the 2013 Physics nobel prize for the theoretical development of the so-called Higgs mechanism.  The recommendations from our system are very good, including the other articles which were also responsible for these theoretical developments as aknowledged by the community.

Another interesting example is this one  

["Unitary Symmetry and Leptonic Decays"](http://www.hep-recommender.com/?article=4510) by Cabibbo, Nicola (1963)

This article introduced the idea of quark mixing when only two quark generations were known.  The recommendations include the article

["CP Violation in the Renormalizable Theory of Weak Interaction"](https://labs.inspirehep.net/literature/81350) by Kobayashi, Makoto; Maskawa, Toshihide (1973)

which extends the idea of Cabibbo to three quark generations and gave rise to what is now known as the CKM matrix.

Exploring a large set of articles I was surprised by the high quality of the recommendations.  Several experts on the field of HEP have also been using the application and find the recommendations satisfactory.

# References

[[1]](https://www.semanticscholar.org/paper/Co-citation-in-the-scientific-literature%3A-A-new-of-Small/da30b84925764b550b55c7d00596f8f1b9608fe2) H. Small, “Co-citation in the scientific literature: A new measure of the relationship between two documents” Journal of the American Society for Information Science, vol. 24, no. 4, pp. 265–269, 1973.

[[2]](https://www.semanticscholar.org/paper/Citation-Proximity-Analysis-%28CPA%28-%3A-A-New-Approach-Gipp-Beel/22b519fcf4b623bef5cd702f2e1c995640d4c833) B. Gipp and J. Beel, “Citation proximity analysis (cpa) : A new approach for identifying related work based on co-citation analysis” in Proceedings of the 12th International Confer- ence on Scientometrics and Informetrics, vol. 1 (B. Larsen, ed.), (Sao Paulo), pp. 571–575, BIREME/PANO/WHO, 2009.

[[3]](https://www.semanticscholar.org/paper/A-Scalable-Hybrid-Research-Paper-Recommender-System-Kanakia-Shen/bb246e08bc6641672c2bb2b93d4214eccf3f84b6) A. Kanakia, Z. Shen, D. Eide, and K. Wang, “A scalable hybrid research paper recommender system for microsoft academic” CoRR, vol. abs/1905.08880, 2019.

[[4]](https://www.semanticscholar.org/paper/System-of-Document-Connections-Based-on-References-Marshakova-shaikevich/2d871489eb7288dd1bec4be99bc363efd4933d48) I. V. Marshakova-shaikevich, “System of document connections based on references” 2009.

[[5]](https://www.semanticscholar.org/paper/Bibliographic-coupling-between-scientific-papers-Kessler/68300052245f0f3aed1a4d65943e436cf1227242) M. M. Kessler, “Bibliographic coupling between scientific papers” American Documentation, vol. 14, no. 1, pp. 10–25, 1963.

[[6]](https://www.semanticscholar.org/paper/Efficient-Estimation-of-Word-Representations-in-Mikolov-Chen/330da625c15427c6e42ccfa3b747fb29e5835bf0) T. Mikolov, K. Chen, G. S. Corrado, and J. Dean, “Efficient estimation of word representations in vector space” CoRR, vol. abs/1301.3781, 2013.

[[7]](https://www.semanticscholar.org/paper/Evaluating-link-based-recommendations-for-Wikipedia-Schwarzer-Schubotz/7c72a2008f94078ce50ac251c6437038bbcf0185) M. Schwarzer, M. Schubotz, N. Meuschke, C. Breitinger, V. Markl, and B. Gipp, “Evaluating link-based recommendations for wikipedia” 2016 IEEE/ACM Joint Conference on Digital Libraries (JCDL), pp. 191–200, 2016.